# Coverage of total tests, cases, deaths
Using the owi-covid-data.csv file from https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv

In [ ]:
import requests
import json
import pandas as pd
import warnings
import csv
warnings.filterwarnings('ignore', category=UserWarning, message="Could not infer format")
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import date

import seaborn as sns
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates


# Consider only countries selected from previous filters
dl = pd.read_csv('csv/FULL_DATA3.csv')  
countries= dl['country'].unique().tolist()
dx=pd.read_csv('csv/full_data3_con_usa.csv') # to have total cases for US

In [ ]:
dg = pd.read_csv('csv/owid-covid-data.csv') 
#dg.columns
dg.loc[dg['location'] == "Czechia", 'location'] = "Czech Republic"
dg= dg[dg['location'].isin(countries)]
dg['date'] = pd.to_datetime(dg['date'])
dg['date'] = dg['date'].dt.date
dg= dg[(dg['date'] >= date(2021, 5, 21)) & (dg['date'] <= date(2022, 6, 25))]

dg_filtered= dg[['location', 'date', 'total_cases', 'total_tests','stringency_index' , 'total_deaths']]
#dg_filtered
country_nan= dg_filtered[dg_filtered['total_tests'].isna()]['location'].unique().tolist()
print(country_nan)

In [ ]:
# merge DataFrames

dg_filtered2=dg_filtered.rename(columns={
    'location': 'country',
    'date': 'survey_date',
})
dx['survey_date'] = pd.to_datetime(dx['survey_date'])
dg_filtered2['survey_date'] = pd.to_datetime(dg_filtered2['survey_date'])
dg_filtered2=dg_filtered2[['country',	'survey_date',	'total_cases',	'total_tests']]
dx.set_index(["country", "survey_date"], inplace=True)
dg_filtered2.set_index(["country", "survey_date"], inplace=True)
dx.update(dg_filtered2[["total_cases", "total_tests"]]) # aggiorna solo le colonne presenti in df1
dx.reset_index(inplace=True)   # torna al formato originale
dg_filtered2.reset_index(inplace=True)

#dx contains  all tests: world + US

In [ ]:
# Prepare data
dg_filtered_subset = dx[['country', 'survey_date', 'total_tests']].copy()
dg_filtered_subset['is_nan'] = dg_filtered_subset['total_tests'].isna().astype(int)

pivot_table = dg_filtered_subset.pivot(index='country', columns='survey_date', values='is_nan')
pivot_table = pivot_table.reindex(sorted(pivot_table.columns), axis=1)


In [ ]:
plt.figure(figsize=(15, 22))
plt.rcParams['font.family'] = 'sans-serif' 
sns.heatmap(pivot_table, cmap='Greys', cbar=False)

legend_patch = Patch(color='black', label='NaN in total_tests')
plt.legend(handles=[legend_patch], loc='upper center', bbox_to_anchor=(0.80, 1.07), ncol=1, frameon=False, fontsize=16)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Country', fontsize=16)
plt.title('Occurrence of NaN in total_tests by Country', fontsize=18)

# Axes tick labels
dates = pivot_table.columns
ticks = [i for i, date in enumerate(dates) if date.day == 1]
if 0 not in ticks:
    ticks.insert(0, 0)
if (len(dates) - 1) not in ticks:
    ticks.append(len(dates) - 1)

# Labels
labels = [dates[i].strftime('%Y-%m-%d') for i in ticks]

plt.xticks(ticks=ticks, labels=labels, fontsize=14,  ha='right')
plt.grid(True, axis='y', linestyle='-', alpha=0.5)
plt.yticks(fontsize=16)

plt.tight_layout()
plt.show()


## total cases e total deaths 

In [ ]:
dg_filtere_subset = dx[['country', 'survey_date', 'total_cases']].copy()
dg_filtere_subset['is_nan'] = dg_filtere_subset['total_cases'].isna().astype(int)

p_table = dg_filtere_subset.pivot(index='country', columns='survey_date', values='is_nan')
p_table = p_table.reindex(sorted(p_table.columns), axis=1)

# Plot
plt.figure(figsize=(20, 10))
sns.heatmap(p_table, cmap='Greys', cbar=False)  # No colorbar

legend_patch = Patch(color='black', label='NaN in total cases')
plt.legend(handles=[legend_patch], loc='upper center', bbox_to_anchor=(0.80, 1.07), ncol=1, frameon=False)
plt.xlabel('date')
plt.ylabel('country')
plt.title('Occurrence of NaN in total cases by country')
plt.tight_layout()
plt.show()

In [ ]:
dg_filtere_subset = dg_filtered[['location', 'date', 'total_deaths']].copy()
dg_filtere_subset['is_nan'] = dg_filtere_subset['total_deaths'].isna().astype(int)

p_table = dg_filtere_subset.pivot(index='location', columns='date', values='is_nan')
p_table = p_table.reindex(sorted(p_table.columns), axis=1)

# Plot
plt.figure(figsize=(20, 10))
sns.heatmap(p_table, cmap='Greys', cbar=False)  # No colorbar

legend_patch = Patch(color='black', label='NaN in total deaths ')
plt.legend(handles=[legend_patch], loc='upper center', bbox_to_anchor=(0.80, 1.07), ncol=1, frameon=False)
plt.xlabel('date')
plt.ylabel('country')
plt.title('Occurrence of NaN in total deaths by country')
plt.tight_layout()
plt.show()

In [ ]:
# Exclude countries with too big gaps in tot_test, cases and deaths
cut_list=['Brazil', 'Egypt', 'Ukraine', 'Venezuela', 'Vietnam', 'Taiwan']

full_data = pd.read_csv("csv/FULL_DATA3.csv")
filtered_full_data = full_data[~full_data["country"].isin(cut_list)]
filtered_full_data.to_csv("csv/FULL_DATA4.csv", index=False)

## Compare share of positive tests and total tests, for uncertain countries 

https://ourworldindata.org/grapher/positive-rate-daily-smoothed

In [ ]:
cut_listt= ['Brazil',  'Ukraine', 'Vietnam']

dts = pd.read_csv('csv/positive-rate-daily-smoothed.csv')
dts= dts[dts['Entity'].isin(cut_listt)]
dts['Day'] = pd.to_datetime(dts['Day'])
dts['Day'] = dts['Day'].dt.date
dts= dts[(dts['Day'] >= date(2021, 5, 21)) & (dts['Day'] <= date(2022, 6, 25))]
dts

In [ ]:
from matplotlib.ticker import FuncFormatter

def format_func(value, _):
    if value >= 1e6:
        return f"{int(value/1e6)}M"
    elif value >= 1e3:
        return f"{int(value/1e3)}K"
    return str(int(value))

# Plot
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 3))  
axs = axs.flatten() 

for idx, country in enumerate(cut_listt[:5]): 
    dg_country = dg[dg["location"] == country]
    dts_country = dts[dts["Entity"] == country]
    
    ax1 = axs[idx]
    ax1.yaxis.set_major_formatter(FuncFormatter(format_func))
    plt.rcParams['font.size'] = 10  
    date_format = "%Y-%m"  
  
    ax1.plot(
        pd.to_datetime(dg_country["date"]),
        dg_country["total_tests"],
        color="tab:blue",
        label="Total tests"
    )
    ax1.set_xlabel("Date", fontsize=9)
    ax1.set_ylabel("Total tests", color="tab:blue", fontsize=10)
    ax1.tick_params(axis="y", labelcolor="tab:blue")
    ax1.xaxis.set_major_formatter(mdates.DateFormatter(date_format))  
    ax1.tick_params(axis='x', labelsize=9, rotation=45) 
    
   
    ax2 = ax1.twinx()
    ax2.plot(
        pd.to_datetime(dts_country["Day"]),
        dts_country["COVID-19 positivity rate"],
        color="tab:red",
        label="Share of positive test"
    )
    ax2.set_ylabel("Share of positive test (%)", color="tab:red", fontsize=10)
    ax2.tick_params(axis="y", labelcolor="tab:red")
    ax2.set_ylim(0, 100)
    ax1.set_title(f"{country}", fontsize=12)
    ax1.set_xlim([dg_country['date'].min(), dg_country['date'].max()])

plt.tight_layout()
plt.show()


## Intervention indicators
Facial covering,  Stringency Index, CHI, goverment response index: https://ourworldindata.org/grapher/face-covering-policies-covid
e https://ourworldindata.org/grapher/covid-containment-and-health-index?tab=table#sources-and-processing

In [ ]:
# Face covering

ds = pd.read_csv('csv/face-covering-policies-covid.csv')

cut_list=['Brazil', 'Egypt', 'Ukraine', 'Venezuela', 'Vietnam', 'Taiwan']

ds.loc[ds['Entity'] == "Czechia", 'Entity'] = "Czech Republic"
ds= ds[ds['Entity'].isin(countries)]
ds= ds[~ds['Entity'].isin(cut_list)]
ds['Day'] = pd.to_datetime(ds['Day'])
ds['Day'] = ds['Day'].dt.date
ds= ds[(ds['Day'] >= date(2021, 5, 21)) & (ds['Day'] <= date(2022, 6, 25))]
ds
# verify tthat face covering and H6 are equal
#argentina_df = df[df['CountryName'] == 'Argentina']['H6M_Facial Coverings']  
#argentina_ds = ds[ds['Entity'] == 'Argentina']['Facial Coverings']  
#print(argentina_df)
#print(argentina_ds)

In [ ]:
# CHI

dv = pd.read_csv('csv/covid-containment-and-health-index.csv')
dv.loc[dv['Entity'] == "Czechia", 'Entity'] = "Czech Republic"
dv= dv[dv['Entity'].isin(countries)]
dv= dv[~dv['Entity'].isin(cut_list)]
dv['Day'] = pd.to_datetime(dv['Day'])
dv['Day'] = dv['Day'].dt.date
dv= dv[(dv['Day'] >= date(2021, 5, 21)) & (dv['Day'] <= date(2022, 6, 25))]
dv

In [ ]:
dg= dg[~dg['location'].isin(cut_list)]
dg
# verify that CHI-OWID and CHI-OXFORD are equal (they are!)
#argentina_df = df[df['CountryName'] == 'Argentina']['ContainmentHealthIndex_Average']  
#argentina_dv = dv[dv['Entity'] == 'Argentina']['Containment health index (average)']  
#print(argentina_df)
#print(argentina_dv)

In [ ]:
# Government response index; dowload file from https://github.com/OxCGRT/covid-policy-dataset/tree/main/data/timeseries_indices

dr = pd.read_csv('csv/OxCGRT_timeseries_GovernmentResponseIndex_v1.csv')
dr= dr[dr['Jurisdiction'] == 'NAT_TOTAL']

dr= dr[dr['CountryName'].isin(countries)]
dr= dr[~dr['CountryName'].isin(cut_list)]
dr= dr[~dr['CountryName'].isin(cut_list)]
dr = dr.drop(dr[dr['CountryName'] == 'Georgia'].index)

dr = dr.sort_values(by='CountryName', ascending=True)

# dates I need: from column 513 to 913
dr

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Lista dei paesi
countries = ['Argentina', 'Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Colombia',
 'Czech Republic', 'Denmark', 'Ecuador', 'Finland', 'France', 'Germany',
 'Greece', 'Hungary', 'Indonesia', 'Italy', 'Japan' ]
n_countries = len(countries)

n_per_row = 3  # <-- Cambiato da 5 a 4
n_rows = int(np.ceil(n_countries / n_per_row))

plt.figure(figsize=(15, 20)) 

for idx, country in enumerate(countries, 1):
    dg_country = dg[dg["location"] == country]
    ds_country = ds[ds["Entity"] == country]
    dv_country = dv[dv["Entity"] == country]
    dr_country = dr[dr['CountryName'] == country]

    ax1 = plt.subplot(n_rows, n_per_row, idx)

    # Plot 1 - Stringency Index
    ax1.plot(
        pd.to_datetime(dg_country["date"]),
        dg_country["stringency_index"],
        color="tab:blue",
        label="Stringency Index",
        linewidth=1
    )
    # Plot 2 - Containment & Health Index
    ax1.plot(
        pd.to_datetime(dv_country["Day"]),
        dv_country["Containment health index (average)"],
        color="tab:green",
        label="Containment and Health Index",
        linewidth=1,
        alpha=0.7
    )

    # Plot 3 - Government Intervention Index
    ax1.plot(
        pd.to_datetime(dv_country["Day"]),
        dr_country.iloc[:,513:914].values.flatten(),
        color="tab:grey",
        label="Government Intervention Index",
        linewidth=1,
        alpha=0.7
    )

    # Secondo asse y per Face Coverage
    ax2 = ax1.twinx()
    ax2.plot(
        pd.to_datetime(ds_country["Day"]),
        ds_country["Facial Coverings"],
        color="tab:red",
        label="Face Coverage",
        linestyle="--",
        linewidth=1
    )

    ax1.set_title(country, fontsize=16)
    ax1.tick_params(axis='x', labelsize=7, rotation=45)
    ax1.tick_params(axis='y', labelsize=7)
    ax2.tick_params(axis='y', labelsize=7)

    # Etichette Y solo nella prima colonna
    ax1.set_ylabel('Stringency / Containment /\n Intervention', fontsize=11)
    ax2.set_ylabel('Face Coverage', fontsize=11)
    ax1.set_xlabel('Date', fontsize=11)


# Legenda globale
lines = [
    plt.Line2D([0], [0], color='tab:blue', lw=1, label='Stringency Index'),
    plt.Line2D([0], [0], color='tab:green', lw=1, label='Containment and Health Index'),
    plt.Line2D([0], [0], color='tab:grey', lw=1, label='Government Intervention Index'),
    plt.Line2D([0], [0], color='tab:red', lw=1, linestyle='--', label='Face Coverage')
]
plt.figlegend(handles=lines, loc='lower center', ncol=4, fontsize=14)

plt.tight_layout(rect=[0, 0.05, 1, 0.98])
plt.show()

In [ ]:

countries = ['Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Peru', 'Philippines', 'Poland',
 'Portugal', 'Romania', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Turkey', 'United Kingdom']
n_countries = len(countries)

n_per_row = 3 
n_rows = int(np.ceil(n_countries / n_per_row))
plt.figure(figsize=(15, 20)) 

for idx, country in enumerate(countries, 1):
    dg_country = dg[dg["location"] == country]
    ds_country = ds[ds["Entity"] == country]
    dv_country = dv[dv["Entity"] == country]
    dr_country = dr[dr['CountryName'] == country]

    ax1 = plt.subplot(n_rows, n_per_row, idx)

    # Plot 1 - Stringency Index
    ax1.plot(
        pd.to_datetime(dg_country["date"]),
        dg_country["stringency_index"],
        color="tab:blue",
        label="Stringency Index",
        linewidth=1
    )
    # Plot 2 - Containment & Health Index
    ax1.plot(
        pd.to_datetime(dv_country["Day"]),
        dv_country["Containment health index (average)"],
        color="tab:green",
        label="Containment and Health Index",
        linewidth=1,
        alpha=0.7
    )

    # Plot 3 - Government Intervention Index
    ax1.plot(
        pd.to_datetime(dv_country["Day"]),
        dr_country.iloc[:,513:914].values.flatten(),
        color="tab:grey",
        label="Government Intervention Index",
        linewidth=1,
        alpha=0.7
    )

    # Secondo asse y per Face Coverage
    ax2 = ax1.twinx()
    ax2.plot(
        pd.to_datetime(ds_country["Day"]),
        ds_country["Facial Coverings"],
        color="tab:red",
        label="Face Coverage",
        linestyle="--",
        linewidth=1
    )

    ax1.set_title(country, fontsize=16)
    ax1.tick_params(axis='x', labelsize=7, rotation=45)
    ax1.tick_params(axis='y', labelsize=7)
    ax2.tick_params(axis='y', labelsize=7)

    # Etichette Y solo nella prima colonna
    ax1.set_ylabel('Stringency / Containment /\n Intervention', fontsize=11)
    ax2.set_ylabel('Face Coverage', fontsize=11)
    ax1.set_xlabel('Date', fontsize=11)


# Legenda globale
lines = [
    plt.Line2D([0], [0], color='tab:blue', lw=1, label='Stringency Index'),
    plt.Line2D([0], [0], color='tab:green', lw=1, label='Containment and Health Index'),
    plt.Line2D([0], [0], color='tab:grey', lw=1, label='Government Intervention Index'),
    plt.Line2D([0], [0], color='tab:red', lw=1, linestyle='--', label='Face Coverage')
]
plt.figlegend(handles=lines, loc='lower center', ncol=4, fontsize=14)

plt.tight_layout(rect=[0, 0.05, 1, 0.98])
plt.show()

### Update database

In [ ]:
full_data = pd.read_csv("csv/FULL_DATA4.csv")

# add CHI to the database 
dg_filtered['date'] = pd.to_datetime(dg_filtered['date']).dt.date
dv['Day'] = pd.to_datetime(dv['Day']).dt.date

dg_filtered = pd.merge(
    dg_filtered,
    dv[['Entity', 'Day', 'Containment health index (average)']],
    left_on=['location', 'date'],
    right_on=['Entity', 'Day'],
    how='left'  
)

dg_filtered.rename(columns={'Containment health index (average)': 'Containment health index'}, inplace=True)
dg_filtered.drop(['Entity', 'Day'], axis=1, inplace=True)  # Rimuovi colonne temporanee
dg_filtered

In [ ]:
# merge to FUll data
filtered_full_data['survey_date'] = pd.to_datetime(filtered_full_data['survey_date']).dt.date

# rename columns
cols_to_add = ['total_cases', 'total_tests', 'total_deaths','Containment health index']
dg_selected = dg_filtered[['location', 'date'] + cols_to_add].rename(
    columns={'location': 'country', 'date': 'survey_date'}
)

# merge dataframes
full_data_merged = pd.merge(
    filtered_full_data,
    dg_selected,
    on=['country', 'survey_date'],
    how='left'
)


df_full = full_data_merged

dg_renamed = dg.rename(columns={'location': 'country', 'date': 'survey_date'})

df_full['survey_date'] = pd.to_datetime(df_full['survey_date'], errors='coerce')
dg_renamed['survey_date'] = pd.to_datetime(dg_renamed['survey_date'], errors='coerce')
df_full['country'] = df_full['country'].str.strip().str.lower()
dg_renamed['country'] = dg_renamed['country'].str.strip().str.lower()


df_full.set_index(['country', 'survey_date'], inplace=True)
dg_renamed.set_index(['country', 'survey_date'], inplace=True)

# add columns
df_full = df_full.join(dg_renamed[['new_cases_smoothed', 'new_deaths_smoothed']], how='left')
df_full.reset_index(inplace=True) 

df_full['country'] = df_full['country'].str.title()
df_full.to_csv('csv/FULL_DATA5_FINALE.csv', index=False)
df_full

## Ranking of countries by government interventions

In [ ]:
# Read files from Oxford GitHub “A global panel database of pandemic policies (Oxford COVID-19 Government Response Tracker).” Nature Human Behaviour. https://doi.org/10.1038/s41562-021-01079-8
url = "https://github.com/OxCGRT/covid-policy-dataset/blob/main/data/OxCGRT_compact_national_v1.csv?raw=true"
df = pd.read_csv(url)
df.head()
#df.columns

In [ ]:
# Consider only our countries
dl = pd.read_csv('csv/facebook_data_2021.csv')  
countries= dl['country'].unique().tolist()

df= df[df['CountryName'].isin(countries)]
# Convert dates in datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Date'] = df['Date'].dt.date # no hours
df = df[(df['Date'] >= date(2021, 5, 21)) & (df['Date'] <= date(2022, 6, 25))]

cut_list=['Brazil', 'Egypt', 'Ukraine', 'Venezuela', 'Vietnam', 'Taiwan', 'India']
df = df[~df['CountryName'].isin(cut_list)]


In [ ]:
# Group countries, average of stringencyindex_average, min and max for each country 
statistiche = df.groupby('CountryName')['ContainmentHealthIndex_Average'].agg(['mean', 'min', 'max']).reset_index()
statistiche.columns = ['CountryName', 'MeanCHI', 'MinCHI', 'MaxCHI'] # rename columns
mean_stringency = statistiche

print(mean_stringency.head())

In [ ]:
# Shapiro-Wilk test
shapiro_stat, shapiro_p = stats.shapiro(mean_stringency['MeanCHI'])

# Histo
plt.figure(figsize=(13, 5))
plt.subplot(1, 2, 1)
sns.histplot(mean_stringency['MeanCHI'], kde=True, stat='density', bins=15)
plt.title('Distribution of \n Mean Containment and Health Index')
plt.xlabel('Mean Containment and Health Index')
plt.ylabel('Density')

# Add Shapiro-Wilk result in the histogram
plt.annotate(
    f'Shapiro-Wilk\nW = {shapiro_stat:.3f}\np = {shapiro_p:.3f}',
    xy=(0.05, 0.95),
    xycoords='axes fraction',
    ha='left',
    va='top',
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray')
)
# Q-Q plot
plt.subplot(1, 2, 2)
stats.probplot(mean_stringency['MeanCHI'], dist='norm', plot=plt)
plt.title('Q-Q Plot Against Normal Distribution \n')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.tight_layout()
plt.show()


In [ ]:
# Statistical test for normality: Shapiro-Wilk 
from scipy.stats import shapiro

stat, p_value = shapiro(mean_stringency['MeanCHI'])
print(f"Shapiro-Wilk Test: p-value = {p_value:.4f}")
if p_value > 0.05:
    print("Cannot refuse null hypothesis: the distribution is norma.")
else:
    print("Refuse null hypothesis: distribution is NOT normal.")

In [ ]:
#skewness (asymmetry) and kurtosis
# skewness ≈ 0: symmetric distribution (like Gaussian) ->   < 0: long left tail
# kurtosis ≈ 3: tails close to Gaussian.

from scipy.stats import skew, kurtosis

skewness = skew(mean_stringency['MeanCHI'])
kurt = kurtosis(mean_stringency['MeanCHI'])

print(f"Skewness: {skewness:.2f} (close to 0 = symmetry)")
print(f"Kurtosis: {kurt:.2f} (close to 3 = not peaked)")

In [ ]:
# threshold to divide countries into "more or less interventist"-> use mean of dataset as it's Gaussian
threshold = mean_stringency['MeanCHI'].mean()
print(threshold)
mean_stringency['Intervention'] = mean_stringency['MeanCHI'].apply(lambda x: 'interventista' if x >= threshold else 'non interventista')# Assegna label
print(mean_stringency.head())

In [ ]:

mean_stringency_sorted = mean_stringency.sort_values('MeanCHI') # order df
# array of colors according to threshold
colors = np.where(mean_stringency_sorted['MeanCHI'] >= threshold, 'green', 'red')
labels = np.where(mean_stringency_sorted['MeanCHI'] >= threshold, 'More Interventionist', 'Less Interventionist')

plt.figure(figsize=(10, 8))
bars = plt.barh(mean_stringency_sorted['CountryName'], 
                mean_stringency_sorted['MeanCHI'], 
                color=colors, 
                alpha=0.7)

# Threshold line
plt.axvline(threshold, color='black', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.1f}')
plt.xlim(0, 100)

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
legend_elements = [
    Patch(facecolor='green', alpha=0.7, label='Interventionist'),
    Patch(facecolor='red', alpha=0.7, label='Mildly Interventionist'),
    Line2D([0], [0], color='black', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.1f}') 
]
plt.legend(handles=legend_elements)


plt.xlabel('Mean Containment and Health Index', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Country Ranking by Intervention Level: National Mean VS Global Threshold', fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))

# Histogram for MinCHI (red)
sns.histplot(
    mean_stringency['MinCHI'], 
    color='red', 
    kde=True, 
    stat='density', 
    bins=15, 
    alpha=0.5, 
    label='Minimum CHI per Country'
)

# Histogram for MaxCHI (blue)
sns.histplot(
    mean_stringency['MaxCHI'], 
    color='blue', 
    kde=True, 
    stat='density', 
    bins=15, 
    alpha=0.5, 
    label='Maximum CHI per Country'
)

# Threshold line
plt.axvline(
    threshold, 
    color='black', 
    linestyle='-', 
    linewidth=3, 
    label=f'Selected Threshold: {threshold:.1f}'
)

plt.title('Distributions of Minimum and Maximum Containment and Health Index Across Countries', fontsize=14)
plt.xlabel('Containment and Health Index (CHI)')
plt.ylabel('Density')
plt.legend(
    loc='upper center',
    bbox_to_anchor=(0.5, -0.2),
    ncol=3, 
    frameon=False
)

plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.subplots_adjust(bottom=0.3) 
plt.show()



In [ ]:
minsum=mean_stringency['MinCHI'].mean()
maxsum=mean_stringency['MaxCHI'].mean()
print(minsum)
print(maxsum)
mean_stringency.to_csv("csv/interventionCHI.csv", index=False, encoding="utf-8")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
fig.suptitle('Country Intervention Levels: Minimum vs Maximum Values', fontsize=20, y=1.02)

# Calculate thresholds
min_threshold = mean_stringency['MinCHI'].mean()
max_threshold = mean_stringency['MaxCHI'].mean()

# Order data
min_sorted = mean_stringency.sort_values('MinCHI')
max_sorted = mean_stringency.sort_values('MaxCHI')

country_fontsize = 16
axis_fontsize = 16

# Subplot 1: Min
colors_min = np.where(min_sorted['MinCHI'] >= min_threshold, 'green', 'red')
ax1.barh(min_sorted['CountryName'], min_sorted['MinCHI'], color=colors_min, alpha=0.7, height=0.8)
ax1.axvline(min_threshold, color='black', linestyle='--', linewidth=2, label=f'Threshold: {min_threshold:.1f}')
ax1.set_xlim(0, 100)
ax1.set_title('Minimum Intervention Levels', fontsize=18, pad=13)
ax1.set_xlabel('Minimum Containment and Health Index', fontsize=axis_fontsize)
ax1.set_ylabel('Country', fontsize=axis_fontsize)
ax1.tick_params(axis='y', labelsize=country_fontsize)
ax1.tick_params(axis='x', labelsize=14)
ax1.grid(axis='x', linestyle='--', alpha=0.4)
ax1.legend(loc='lower right', fontsize=14)  # Legenda individuale per ax1

# Subplot 2: Max
colors_max = np.where(max_sorted['MaxCHI'] >= max_threshold, 'green', 'red')
ax2.barh(max_sorted['CountryName'], max_sorted['MaxCHI'], color=colors_max, alpha=0.7, height=0.8)
ax2.axvline(max_threshold, color='black', linestyle='--', linewidth=2, label=f'Threshold: {max_threshold:.1f}')
ax2.set_xlim(0, 100)
ax2.set_title('Maximum Intervention Levels', fontsize=18, pad=13)
ax2.set_xlabel('Maximum Containment and Health Index', fontsize=axis_fontsize)
ax2.tick_params(axis='x', labelsize=14)
ax2.grid(axis='x', linestyle='--', alpha=0.4)
ax2.tick_params(axis='y', labelsize=country_fontsize)
ax2.legend(loc='lower right', fontsize=14)  # Legenda individuale per ax2

# Legend for colors
legend_elements = [
    Patch(facecolor='green', alpha=0.7, label='Interventionist (≥ threshold)'),
    Patch(facecolor='red', alpha=0.7, label='Mildly Interventionist (< threshold)')
]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, fontsize=16)

plt.tight_layout()

plt.show()
